# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.78,93,100,5.13,TF,1712806946
1,1,saint-philippe,-21.3585,55.7679,24.77,79,75,4.13,RE,1712806947
2,2,hawaiian paradise park,19.5933,-154.9731,23.73,87,100,3.60,US,1712806947
3,3,male,4.1748,73.5089,31.97,74,40,2.57,MV,1712806948
4,4,palu,-0.8917,119.8707,31.65,69,63,1.53,ID,1712806949


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
# Define a color palette for each city based on a categorical column (e.g., 'City')
map_plot_humidity = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    color='City',  # Assign a unique color to each city
    cmap='Category10',  # Use a categorical color palette
    legend='top_right',  # Position the legend to the top left
)
# Display the map plot
map_plot_humidity


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null value
#cities with humidity less than or equal to 55
ideal_weather_df = city_data_df.loc[city_data_df["Humidity"] >= 55]
# Drop any rows with null values
ideal_weather_df.dropna()
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,6.78,93,100,5.13,TF,1712806946
1,1,saint-philippe,-21.3585,55.7679,24.77,79,75,4.13,RE,1712806947
2,2,hawaiian paradise park,19.5933,-154.9731,23.73,87,100,3.60,US,1712806947
3,3,male,4.1748,73.5089,31.97,74,40,2.57,MV,1712806948
4,4,palu,-0.8917,119.8707,31.65,69,63,1.53,ID,1712806949
...,...,...,...,...,...,...,...,...,...,...
554,554,jilib,0.4883,42.7854,28.08,78,53,1.83,SO,1712807240
555,555,jorpeland,59.0225,6.0408,9.50,97,100,5.03,NO,1712807241
556,556,tangalle,6.0240,80.7911,31.09,60,94,4.12,LK,1712807242
557,557,ponta delgada,37.7333,-25.6667,18.44,94,20,6.17,PT,1712807214


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[[ "City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,93,
1,saint-philippe,RE,-21.3585,55.7679,79,
2,hawaiian paradise park,US,19.5933,-154.9731,87,
3,male,MV,4.1748,73.5089,74,
4,palu,ID,-0.8917,119.8707,69,
...,...,...,...,...,...,...
554,jilib,SO,0.4883,42.7854,78,
555,jorpeland,NO,59.0225,6.0408,97,
556,tangalle,LK,6.0240,80.7911,60,
557,ponta delgada,PT,37.7333,-25.6667,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row ["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
saint-philippe - nearest hotel: Le Baril
hawaiian paradise park - nearest hotel: No hotel found
male - nearest hotel: Champa Central Hotel
palu - nearest hotel: Ujung Pandang Hotel
betio village - nearest hotel: Seaman Hostel
talnakh - nearest hotel: Талнах
burang - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
hammerfest - nearest hotel: Thon Hotel Hammerfest
bethel - nearest hotel: Hampton Inn Danbury
bubaque - nearest hotel: Chez Julio
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
arraial do cabo - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
general villegas - nearest hotel: Rucalén
edinburgh of the seven seas - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
grytviken - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
wail

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,93,Keravel
1,saint-philippe,RE,-21.3585,55.7679,79,Le Baril
2,hawaiian paradise park,US,19.5933,-154.9731,87,No hotel found
3,male,MV,4.1748,73.5089,74,Champa Central Hotel
4,palu,ID,-0.8917,119.8707,69,Ujung Pandang Hotel
...,...,...,...,...,...,...
554,jilib,SO,0.4883,42.7854,78,No hotel found
555,jorpeland,NO,59.0225,6.0408,97,Verkshotellet
556,tangalle,LK,6.0240,80.7911,60,Coco Tangalla
557,ponta delgada,PT,37.7333,-25.6667,94,Hotel Talisman


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

hotel_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    color='City',
    cmap='Category10',
    legend='top_right',
    hover_cols=['City', 'Hotel Name', 'Country'] 
)
# Display the map plot
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)